<p> This notebook is to create the XML files using the annotations already provided while downloading the dataset</p>
<p> the dataset is from : https://universe.roboflow.com/roboflow-100/bone-fracture-7fylg/dataset/2</p>

In [50]:
# import libraries
import pandas as pd

In [51]:
# function to replace dynamic parts with the given data from the excel sheet
def search_and_replace(file_path, search_word, replace_word):
   with open(file_path, 'r') as file:
      file_contents = file.read()

      updated_contents = file_contents.replace(search_word, replace_word)

   with open(file_path, 'w') as file:
      file.write(updated_contents)
   
   file.close()

In [52]:
# function to add more objects to an image
def add_object(file_path, position, content):
   with open(file_path, 'r') as file:
      file_contents = file.readlines()

      file_contents.insert(position, content)

   #print(file_contents)
   with open(file_path, 'w') as file:
      for line in file_contents:
         file.write(line)
   
   file.close()

In [63]:
# open the annotations excel file to extract the data from it
path = input("Enter path of the file : ")
annotations = pd.read_csv(path)
df = pd.DataFrame(annotations)
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,8_jpg.rf.1a2c67587b5266449e319b6682d062a4.jpg,608,1024,fracture,261,728,407,806
1,8_jpg.rf.1a2c67587b5266449e319b6682d062a4.jpg,608,1024,fracture,178,733,236,783
2,27_jpg.rf.11e9a048a26c4205da3b1ce533cb9113.jpg,1024,1024,messed_up_angle,382,614,502,775
3,87_jpg.rf.0fe6c38fe536952979272ec167087cd6.jpg,378,1024,line,198,707,231,738
4,39_jpg.rf.04a442d960bdbd11a92b54b7b49e01f4.jpg,1023,1023,fracture,418,413,436,475


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  117 non-null    object
 1   width     117 non-null    int64 
 2   height    117 non-null    int64 
 3   class     117 non-null    object
 4   xmin      117 non-null    int64 
 5   ymin      117 non-null    int64 
 6   xmax      117 non-null    int64 
 7   ymax      117 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 7.4+ KB


In [65]:
df.columns

Index(['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'], dtype='object')

In [66]:
# template for xml annotations
file = open('annot_template.txt', 'w')
file.write("""<annotation>
	<folder>fracture-detection</folder>
	<filename>IMAGE_NAME</filename>
	<path>ANNOT_PATH</path>
	<source>
		<database>Unspecified</database>
	</source>
	<size>
		<width>WIDTH</width>
		<height>HEIGHT</height>
		<depth>3</depth>
	</size>
	<object>
		<name>LABEL</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>XMIN</xmin>
			<ymin>YMIN</ymin>
			<xmax>XMAX</xmax>
			<ymax>YMAX</ymax>
		</bndbox>
	</object>
</annotation>
""")
file.close()

file = open('object_template.txt', 'w')
file.write("""	<object>
		<name>LABEL</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>XMIN</xmin>
			<ymin>YMIN</ymin>
			<xmax>XMAX</xmax>
			<ymax>YMAX</ymax>
		</bndbox>
	</object>
""")
file.close()


In [ ]:
# creating the xml annotations
length = len(df['filename'])
existing_file = list()

path = input("Enter Destination path: ")
for i in range(length):
    fname = df.iloc[i][0][:-4]
    fname = fname+".xml"
    #print(fname)

    if fname not in existing_file:
        existing_file.append(fname)

        f1 = open("annot_template.txt", "r")
        lines = f1.readlines()
        #print(lines)

        f2_name = "{}/{}".format(path, fname)
        f2 = open(f2_name, "w")
        for line in lines:
            f2.write(line)

        values = dict()
        values['IMAGE_NAME'] = df.iloc[i][0]
        values['ANNOT_PATH'] = '/{}/{}'.format(path, fname)
        values['WIDTH'] = df.iloc[i][1]
        values['HEIGHT'] = df.iloc[i][2]
        values['LABEL'] = df.iloc[i][3]
        values['XMIN'] = df.iloc[i][4]
        values['YMIN'] = df.iloc[i][5]
        values['XMAX'] = df.iloc[i][6]
        values['YMAX'] = df.iloc[i][7]

        f2.close()

        for k in values.keys():
            location = values['ANNOT_PATH']
            #print(location)
            #print("{} \n{} \n{} \n\n".format(location[1:], k, str(values[k])))
            search_and_replace(location[1:], k, str(values[k]))

        f1.close()

    else:
        f1 = open("object_template.txt", "r")
        lines = f1.readlines()
        #print(lines)

        f2_name = "{}/{}".format(path, fname)
        pos = 24
        for line in lines:
            #print(line)
            add_object(f2_name, pos, line)
            pos += 1

        values = dict()
        values['IMAGE_NAME'] = df.iloc[i][0]
        values['ANNOT_PATH'] = '/{}/{}'.format(path, fname)
        values['WIDTH'] = df.iloc[i][1]
        values['HEIGHT'] = df.iloc[i][2]
        values['LABEL'] = df.iloc[i][3]
        values['XMIN'] = df.iloc[i][4]
        values['YMIN'] = df.iloc[i][5]
        values['XMAX'] = df.iloc[i][6]
        values['YMAX'] = df.iloc[i][7]

        f2.close()

        for k in values.keys():
            location = values['ANNOT_PATH']
            #print(location)
            #print("{} \n{} \n{} \n\n".format(location[1:], k, str(values[k])))
            search_and_replace(location[1:], k, str(values[k]))

        f1.close()


        